## First version

In [ ]:
import rasterio
import numpy as np
from rasterio.transform import rowcol

def knight_moves():
    """ Returns the relative positions of a knight's moves in a grid. """
    moves = [
        (2, 1), (2, -1), (-2, 1), (-2, -1),
        (1, 2), (1, -2), (-1, 2), (-1, -2)
    ]
    return moves

def calculate_knight_cost(raster, row, col):
    """ Calculates the cost of knight's moves for a given cell. """
    moves = knight_moves()
    costs = np.full(len(moves), np.nan)  # Initialize with nan

    for i, (dr, dc) in enumerate(moves):
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < raster.shape[0] and 0 <= new_col < raster.shape[1]:
            costs[i] = raster[new_row, new_col]

    return costs.mean()  # Return the average cost

def get_raster_indices(src, lon, lat):
    """ Convert geographic coordinates to raster indices """
    return rowcol(src.transform, lon, lat)

# Input raster file, geographic coordinates and output file path
raster_file = input("Enter the path to your raster file: ")
longitude = float(input("Enter the longitude of the origin cell: "))
latitude = float(input("Enter the latitude of the origin cell: "))
output_file = input("Enter the path for the output raster file: ")

# Open the raster file and perform calculations
with rasterio.open(raster_file) as src:
    raster = src.read(1)  # Read the first band
    row, col = get_raster_indices(src, longitude, latitude)
    output_raster = np.full(raster.shape, np.nan)  # Initialize output raster

    # Iterate over each cell in the raster
    for r in range(raster.shape[0]):
        for c in range(raster.shape[1]):
            output_raster[r, c] = calculate_knight_cost(raster, r, c)

    # Write the output raster
    with rasterio.open(
        output_file,
        'w',
        driver=src.driver,
        height=src.height,
        width=src.width,
        count=1,
        dtype=output_raster.dtype,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(output_raster, 1)

print("Output raster created:", output_file)


## Second version

In [2]:
import numpy as np
import rasterio
import heapq

def knight_moves():
    moves = [
        (2, 1), (2, -1), (-2, 1), (-2, -1),
        (1, 2), (1, -2), (-1, 2), (-1, -2)
    ]
    return moves

def is_valid_move(row, col, max_row, max_col):
    return 0 <= row < max_row and 0 <= col < max_col

def dijkstra(raster, start_row, start_col):
    max_row, max_col = raster.shape
    dist = np.full(raster.shape, np.inf)
    dist[start_row, start_col] = 0
    queue = [(0, start_row, start_col)]

    while queue:
        cost, row, col = heapq.heappop(queue)
        for dr, dc in knight_moves():
            new_row, new_col = row + dr, col + dc
            if is_valid_move(new_row, new_col, max_row, max_col):
                new_cost = cost + raster[new_row, new_col]
                if new_cost < dist[new_row, new_col]:
                    dist[new_row, new_col] = new_cost
                    heapq.heappush(queue, (new_cost, new_row, new_col))
    return dist

# Input raster file and starting cell coordinates
raster_file = '/home/hector/Documents/SilkRoute/costsurf_jun_500m.tif'
output_file = 'p/home/hector/Documents/SilkRoute/costdist_jun_500m.tif'
longitude = 29.01281689156581  # Replace with your start cell row
latitude = 41.02212656480667  # Replace with your start cell column

# Open the raster file and calculate accumulated costs
with rasterio.open(raster_file) as src:
    raster = src.read(1)
    accumulated_cost = dijkstra(raster, longitude, latitude)

    # Save the accumulated cost raster
    with rasterio.open(
        output_file,
        'w',
        driver=src.driver,
        height=src.height,
        width=src.width,
        count=1,
        dtype=accumulated_cost.dtype,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(accumulated_cost, 1)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Third version

In [3]:
import numpy as np
import rasterio
import heapq
from rasterio.transform import rowcol

def knight_moves():
    moves = [
        (2, 1), (2, -1), (-2, 1), (-2, -1),
        (1, 2), (1, -2), (-1, 2), (-1, -2)
    ]
    return moves

def is_valid_move(row, col, max_row, max_col):
    return 0 <= row < max_row and 0 <= col < max_col

def dijkstra(raster, start_row, start_col):
    max_row, max_col = raster.shape
    dist = np.full(raster.shape, np.inf)
    dist[start_row, start_col] = 0
    queue = [(0, start_row, start_col)]

    while queue:
        cost, row, col = heapq.heappop(queue)
        for dr, dc in knight_moves():
            new_row, new_col = row + dr, col + dc
            if is_valid_move(new_row, new_col, max_row, max_col):
                new_cost = cost + raster[new_row, new_col]
                if new_cost < dist[new_row, new_col]:
                    dist[new_row, new_col] = new_cost
                    heapq.heappush(queue, (new_cost, new_row, new_col))
    return dist

def get_raster_indices(src, lon, lat):
    """ Convert geographic coordinates to raster indices """
    return rowcol(src.transform, lon, lat)

# Input raster file and geographic coordinates for the starting point
raster_file = input("Enter the path to your cost raster file: ")
output_file = input("Enter the path for the outputcost distance raster file: ")
latitude = float(input("Enter the latitude of the starting point: "))
longitude = float(input("Enter the longitude of the starting point: "))

# Open the raster file and perform calculations
with rasterio.open(raster_file) as src:
    raster = src.read(1)  # Read the first band
    start_row, start_col = get_raster_indices(src, longitude, latitude)
    accumulated_cost = dijkstra(raster, start_row, start_col)

    # Save the accumulated cost raster
    with rasterio.open(
        output_file,
        'w',
        driver=src.driver,
        height=src.height,
        width=src.width,
        count=1,
        dtype=accumulated_cost.dtype,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(accumulated_cost, 1)

print("Output raster created:", output_file)


/home/hector/Documents/SilkRoute/costsurf_jun_500m.tif/home/hector/Documents/SilkRoute/costsurf_jun_500m.tif
29.0128168915658129.01281689156581
41.0221265648066741.02212656480667
/home/hector/Documents/SilkRoute/costdist_jun_500m.tif/home/hector/Documents/SilkRoute/costdist_jun_500m.tif


KeyboardInterrupt: 

## Fourth version

In [3]:
import numpy as np
import rasterio
import heapq
from rasterio.transform import rowcol

def movement_options():
    """ Returns the relative positions of a knight's moves and adjacent moves in a grid. """
    # Knight's moves
    knight_moves = [
        (2, 1), (2, -1), (-2, 1), (-2, -1),
        (1, 2), (1, -2), (-1, 2), (-1, -2)
    ]
    # Directly adjacent moves
    adjacent_moves = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

    return knight_moves + adjacent_moves

def calculate_cost(row, col, dr, dc, base_cost):
    """ Calculate cost based on Euclidean distance """
    distance = np.sqrt(dr**2 + dc**2)
    return base_cost * distance

def is_valid_move(row, col, max_row, max_col):
    return 0 <= row < max_row and 0 <= col < max_col

def dijkstra(raster, start_row, start_col):
    max_row, max_col = raster.shape
    dist = np.full(raster.shape, np.inf)
    dist[start_row, start_col] = 0
    queue = [(0, start_row, start_col)]

    while queue:
        cost, row, col = heapq.heappop(queue)
        for dr, dc in movement_options():
            new_row, new_col = row + dr, col + dc
            if is_valid_move(new_row, new_col, max_row, max_col):
                new_cost = cost + calculate_cost(row, col, dr, dc, raster[new_row, new_col])
                if new_cost < dist[new_row, new_col]:
                    dist[new_row, new_col] = new_cost
                    heapq.heappush(queue, (new_cost, new_row, new_col))
    return dist

def get_raster_indices(src, lon, lat):
    """ Convert geographic coordinates to raster indices """
    return rowcol(src.transform, lon, lat)

# Inputs and raster processing
costs_raster = input("Enter the path to your cost raster file: ")
costsAcum_raster = input("Enter the path for the outputcost distance raster file: ")
latitude = float(input("Enter the latitude/north of the starting point: "))
longitude = float(input("Enter the longitude/east of the starting point: "))

with rasterio.open(costs_raster) as src:
    raster = src.read(1)
    start_row, start_col = get_raster_indices(src, longitude, latitude)
    accumulated_cost = dijkstra(raster, start_row, start_col)

    with rasterio.open(
        costsAcum_raster,
        'w',
        driver=src.driver,
        height=src.height,
        width=src.width,
        count=1,
        dtype=accumulated_cost.dtype,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(accumulated_cost, 1)

print("Output raster created:", output_file)


Enter the path to your cost raster file: C:\ProofCost\proof_dtm.tif
Enter the path for the outputcost distance raster file: C:\ProofCost\proof_costsAcum.tif
Enter the latitude of the starting point: 37.2562
Enter the longitude of the starting point: -2.5576
Output raster created: path_to_output_raster_file.tif
